**Dual LSTM Encoder for Dialog Response Generation**

http://www.wildml.com/2016/07/deep-learning-for-chatbots-2-retrieval-based-model-tensorflow/

https://github.com/dennybritz/chatbot-retrieval

https://github.com/rkadlec/ubuntu-ranking-dataset-creator

https://arxiv.org/abs/1506.08909

In [1]:
import tensorflow as tf
tf.VERSION

'1.2.0'

**Estimator**

In [2]:
help(tf.estimator.Estimator)

Help on class Estimator in module tensorflow.python.estimator.estimator:

class Estimator(builtins.object)
 |  Estimator class to train and evaluate TensorFlow models.
 |  
 |  The `Estimator` object wraps a model which is specified by a `model_fn`,
 |  which, given inputs and a number of other parameters, returns the ops
 |  necessary to perform training, evaluation, or predictions.
 |  
 |  All outputs (checkpoints, event files, etc.) are written to `model_dir`, or a
 |  subdirectory thereof. If `model_dir` is not set, a temporary directory is
 |  used.
 |  
 |  The `config` argument can be passed `RunConfig` object containing information
 |  about the execution environment. It is passed on to the `model_fn`, if the
 |  `model_fn` has a parameter named "config" (and input functions in the same
 |  manner). If the `config` parameter is not passed, it is instantiated by the
 |  `Estimator`. Not passing config means that defaults useful for local execution
 |  are used. `Estimator` make

**model_fn**
```text
 |        model_fn: Model function. Follows the signature:
 |      
 |          * Args:
 |      
 |            * `features`: This is the first item returned from the `input_fn`
 |                   passed to `train`, 'evaluate`, and `predict`. This should be a
 |                   single `Tensor` or `dict` of same.
 |            * `labels`: This is the second item returned from the `input_fn`
 |                   passed to `train`, 'evaluate`, and `predict`. This should be a
 |                   single `Tensor` or `dict` of same (for multi-head models). If
 |                   mode is `ModeKeys.PREDICT`, `labels=None` will be passed. If
 |                   the `model_fn`'s signature does not accept `mode`, the
 |                   `model_fn` must still be able to handle `labels=None`.
 |            * `mode`: Optional. Specifies if this training, evaluation or
 |                   prediction. See `ModeKeys`.
 |            * `params`: Optional `dict` of hyperparameters.  Will receive what
 |                   is passed to Estimator in `params` parameter. This allows
 |                   to configure Estimators from hyper parameter tuning.
 |            * `config`: Optional configuration object. Will receive what is passed
 |                   to Estimator in `config` parameter, or the default `config`.
 |                   Allows updating things in your model_fn based on configuration
 |                   such as `num_ps_replicas`, or `model_dir`.
 |      
 |          * Returns:
 |            `EstimatorSpec`
 ```

In [3]:
def dual_encoder(vocab_size,
                 embed_size,
                 hidden_size,
                 input_context,
                 input_context_len,
                 input_utterance,
                 input_utterance_len,
                 targets):

    with tf.variable_scope('embedding'):
        embeddings = tf.get_variable(
            'embeddings',
            shape=(vocab_size, embed_size),
            initializer=tf.random_uniform_initializer(-0.25, 0.25))

        context_embed = tf.nn.embedding_lookup(
            embeddings, input_context, name='context_embed')
        utterance_embed = tf.nn.embedding_lookup(
            embeddings, input_utterance, name='utterance_embed')

        input_embed = tf.concat([context_embed, utterance_embed], axis=0)
        input_length = tf.concat([input_context_len, input_utterance_len], axis=0)
        input_length = tf.reshape(input_length, [-1])

    with tf.variable_scope('rnn'):
        cell = tf.nn.rnn_cell.LSTMCell(
            hidden_size,
            forget_bias=2.0,
            use_peepholes=True,
            state_is_tuple=True)

        outputs, states = tf.nn.dynamic_rnn(
            cell,
            input_embed,
            sequence_length=input_length,
            dtype=tf.float32)

        context_encoding, utterance_encoding = tf.split(
            states.h, num_or_size_splits=2, axis=0)

    with tf.variable_scope('prediction'):
        ct = context_encoding
        rt = utterance_encoding
        M = tf.get_variable(
            'M',
            shape=(hidden_size, hidden_size),
            initializer=tf.truncated_normal_initializer())

        ct_M = tf.matmul(ct, M)
        batch_ct_M = tf.expand_dims(ct_M, axis=2)
        batch_rt = tf.expand_dims(rt, axis=2)
        batch_ct_M_r = tf.matmul(batch_ct_M, batch_rt, transpose_a=True)
        ct_M_r = tf.squeeze(batch_ct_M_r, axis=2)

        b = tf.get_variable(
            'b', shape=(), initializer=tf.zeros_initializer())
        
        logits = ct_M_r + b
        
        probs = tf.sigmoid(logits)

    if targets is None:
        return probs, None

    loss = tf.losses.sigmoid_cross_entropy(multi_class_labels=tf.to_float(targets), logits=logits)
    loss = tf.reduce_mean(loss, name="loss")
    
    return probs, loss

In [4]:
def model_fn(features, labels, mode, params):
    vocab_size = params['vocab_size']
    embed_size = params['embed_size']
    hidden_size = params['hidden_size']

    input_context = features['context']
    input_context_len = features['context_len']
    input_utterance = features['utterance']
    input_utterance_len = features['utterance_len']

    probs, loss = dual_encoder(
        vocab_size,
        embed_size,
        hidden_size,
        input_context,
        input_context_len,
        input_utterance,
        input_utterance_len,
        labels)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        _learning_rate = params['learning_rate']
        _optimizer =  params['optimizer']
        
        train_op = tf.contrib.layers.optimize_loss(
            loss=loss,
            global_step=tf.contrib.framework.get_global_step(),
            learning_rate=_learning_rate,
            clip_gradients=10.0,
            optimizer=_optimizer)
    else:
        train_op = None
    
    return tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=probs,
        loss=loss,
        train_op=train_op)

**Input**

In [5]:
# `tokenizer` function must be defined before restoring the vocabulary object
# (pickle does not serialize functions)
def tokenizer(sentences):
    return (sentence.split() for sentence in sentences)

class VocabularyAdapter:
    
    def __init__(self, vocabulary_bin):
        self._vocab = tf.contrib.learn.preprocessing.VocabularyProcessor.restore(vocabulary_bin)
    
    @property
    def size(self):
        return len(self._vocab.vocabulary_)
    
    @property
    def vector_length(self):
        return self._vocab.max_document_length


def input_features(vector_length):
    return [
        tf.feature_column.numeric_column(
            key='context', shape=vector_length, dtype=tf.int64),
        tf.feature_column.numeric_column(
            key='context_len', shape=1, dtype=tf.int64),
        tf.feature_column.numeric_column(
            key='utterance', shape=vector_length, dtype=tf.int64),
        tf.feature_column.numeric_column(
            key='utterance_len', shape=1, dtype=tf.int64),
        tf.feature_column.numeric_column(
            key='label', shape=1, dtype=tf.int64),
    ]


def input_fn(name, filenames, features, batch_size, num_epochs=None):
    example_features = tf.feature_column.make_parse_example_spec(features)

    batch_example = tf.contrib.learn.read_batch_record_features(
        file_pattern=filenames,
        features=example_features,
        batch_size=batch_size,
        num_epochs=num_epochs,
        name='read_batch_record_features_' + name
    )

    batch_target = batch_example.pop('label')

    return batch_example, batch_target

**Training**

In [6]:
import os

HOME_DIR = 'ubuntu'
DATA_DIR = os.path.join(HOME_DIR, 'data')
VOCAB_BIN = os.path.join(DATA_DIR, 'vocabulary.bin')
TRAIN_TFR = os.path.join(DATA_DIR, 'train.tfrecords')
VALID_TFR = os.path.join(DATA_DIR, 'valid.tfrecords')
TEST_TFR = os.path.join(DATA_DIR, 'test.tfrecords')

if not os.path.isfile(VOCAB_BIN):
    raise Exception('File not found: {}'.format(VOCAB_BIN))

if not os.path.isfile(TRAIN_TFR):
    raise Exception('File not found: {}'.format(TRAIN_TFR))

if not os.path.isfile(VALID_TFR):
    raise Exception('File not found: {}'.format(VALID_TFR))

if not os.path.isfile(TEST_TFR):
    raise Exception('File not found: {}'.format(TEST_TFR))

In [7]:
import shutil

def remove_dir(path):
    if os.path.isdir(path):
        shutil.rmtree(path)

MODEL_DIR = os.path.join(HOME_DIR, 'model')

remove_dir(MODEL_DIR)

In [8]:
vocab = VocabularyAdapter(VOCAB_BIN)

In [9]:
params = {
    'vocab_size': vocab.size,
    'embed_size': 100,
    'hidden_size': 256,
    'learning_rate': 0.001,
    'optimizer': 'Adam',
    'batch_size': 128,
    'num_epochs': 2,
}

In [10]:
features = input_features(vocab.vector_length)
batch_size = params['batch_size']
num_epochs = params['num_epochs']

input_fn_train = lambda: input_fn('train', [TRAIN_TFR], features, batch_size, num_epochs)
input_fn_valid = lambda: input_fn('valid', [VALID_TFR], features, 32, 1)
input_fn_test = lambda: input_fn('test', [TEST_TFR], features, 32, 1)

In [11]:
estimator = tf.estimator.Estimator(
    model_fn=model_fn,
    model_dir=MODEL_DIR,
    params=params)

estimator

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'ubuntu/model', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000}


In [12]:
%%time

# estimator.train(input_fn=input_fn_train, steps=1001)
estimator.train(input_fn=input_fn_train, steps=None)

INFO:tensorflow:logits.dtype=<dtype: 'float32'>.
INFO:tensorflow:multi_class_labels.dtype=<dtype: 'float32'>.
INFO:tensorflow:losses.dtype=<dtype: 'float32'>.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into ubuntu/model/model.ckpt.
INFO:tensorflow:loss = 0.896316, step = 1
INFO:tensorflow:global_step/sec: 0.711419
INFO:tensorflow:loss = 0.681102, step = 101 (140.566 sec)
INFO:tensorflow:global_step/sec: 0.711863
INFO:tensorflow:loss = 0.719473, step = 201 (140.476 sec)
INFO:tensorflow:global_step/sec: 0.71131
INFO:tensorflow:loss = 0.6546, step = 301 (140.587 sec)
INFO:tensorflow:global_step/sec: 0.711669
INFO:tensorflow:loss = 0.655473, step = 401 (140.514 sec)
INFO:tensorflow:Saving checkpoints for 428 into ubuntu/model/model.ckpt.
INFO:tensorflow:global_step/sec: 0.709543
INFO:tensorflow:loss = 0.626053, step = 501 (140.935 sec)
INFO:tensorflow:global_step/sec: 0.711691
INFO:tensorflow:loss = 0.6248, step = 601 (140.512 sec)
INFO:tensorflow:

INFO:tensorflow:loss = 0.500292, step = 6801 (140.530 sec)
INFO:tensorflow:Saving checkpoints for 6833 into ubuntu/model/model.ckpt.
INFO:tensorflow:global_step/sec: 0.710237
INFO:tensorflow:loss = 0.458765, step = 6901 (140.798 sec)
INFO:tensorflow:global_step/sec: 0.711761
INFO:tensorflow:loss = 0.478114, step = 7001 (140.496 sec)
INFO:tensorflow:global_step/sec: 0.711525
INFO:tensorflow:loss = 0.417596, step = 7101 (140.543 sec)
INFO:tensorflow:global_step/sec: 0.711664
INFO:tensorflow:loss = 0.467996, step = 7201 (140.515 sec)
INFO:tensorflow:Saving checkpoints for 7260 into ubuntu/model/model.ckpt.
INFO:tensorflow:global_step/sec: 0.710176
INFO:tensorflow:loss = 0.479495, step = 7301 (140.811 sec)
INFO:tensorflow:global_step/sec: 0.711639
INFO:tensorflow:loss = 0.423785, step = 7401 (140.521 sec)
INFO:tensorflow:global_step/sec: 0.711543
INFO:tensorflow:loss = 0.46031, step = 7501 (140.539 sec)
INFO:tensorflow:global_step/sec: 0.711418
INFO:tensorflow:loss = 0.464887, step = 7601 

INFO:tensorflow:global_step/sec: 0.709939
INFO:tensorflow:loss = 0.302359, step = 13701 (140.857 sec)
INFO:tensorflow:global_step/sec: 0.711511
INFO:tensorflow:loss = 0.336253, step = 13801 (140.546 sec)
INFO:tensorflow:global_step/sec: 0.711351
INFO:tensorflow:loss = 0.402984, step = 13901 (140.578 sec)
INFO:tensorflow:global_step/sec: 0.711398
INFO:tensorflow:loss = 0.456657, step = 14001 (140.569 sec)
INFO:tensorflow:Saving checkpoints for 14092 into ubuntu/model/model.ckpt.
INFO:tensorflow:global_step/sec: 0.710024
INFO:tensorflow:loss = 0.420469, step = 14101 (140.840 sec)
INFO:tensorflow:global_step/sec: 0.711363
INFO:tensorflow:loss = 0.35533, step = 14201 (140.575 sec)
INFO:tensorflow:global_step/sec: 0.711328
INFO:tensorflow:loss = 0.298576, step = 14301 (140.581 sec)
INFO:tensorflow:global_step/sec: 0.711243
INFO:tensorflow:loss = 0.615326, step = 14401 (140.599 sec)
INFO:tensorflow:global_step/sec: 0.711302
INFO:tensorflow:loss = 0.479245, step = 14501 (140.588 sec)
INFO:ten

In [13]:
%%time

estimator.evaluate(input_fn=input_fn_valid, steps=None)

INFO:tensorflow:logits.dtype=<dtype: 'float32'>.
INFO:tensorflow:multi_class_labels.dtype=<dtype: 'float32'>.
INFO:tensorflow:losses.dtype=<dtype: 'float32'>.
INFO:tensorflow:Starting evaluation at 2017-06-24-22:34:13
INFO:tensorflow:Restoring parameters from ubuntu/model/model.ckpt-14666
INFO:tensorflow:Finished evaluation at 2017-06-24-22:45:56
INFO:tensorflow:Saving dict for global step 14666: global_step = 14666, loss = 0.446515
CPU times: user 20min 28s, sys: 1min 40s, total: 22min 8s
Wall time: 11min 43s


{'global_step': 14666, 'loss': 0.44651476}